In [ ]:
!pip install pygame
!pip install python-chess scikit-learn

In [ ]:
import pygame
import chess
import chess.pgn
import random
import sys
import io
import time
import json  

pygame.init()
tile_size = 80
width, height = tile_size * 8, tile_size * 8
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Chess Game")

piece_images = {
    "P": pygame.transform.scale(pygame.image.load("beyaz_piyon.png"), (tile_size, tile_size)),
    "Q": pygame.transform.scale(pygame.image.load("beyaz_vezir.png"), (tile_size, tile_size)),
    "R": pygame.transform.scale(pygame.image.load("beyaz_kale.png"), (tile_size, tile_size)),
    "B": pygame.transform.scale(pygame.image.load("beyaz_fil.png"), (tile_size, tile_size)),
    "N": pygame.transform.scale(pygame.image.load("beyaz_at.png"), (tile_size, tile_size)),
    "K": pygame.transform.scale(pygame.image.load("beyaz_şah.png"), (tile_size, tile_size)),
    "p": pygame.transform.scale(pygame.image.load("siyah_piyon.png"), (tile_size, tile_size)),
    "q": pygame.transform.scale(pygame.image.load("siyah_vezir.png"), (tile_size, tile_size)),
    "r": pygame.transform.scale(pygame.image.load("siyah_kale.png"), (tile_size, tile_size)),
    "b": pygame.transform.scale(pygame.image.load("siyah_fil.png"), (tile_size, tile_size)),
    "n": pygame.transform.scale(pygame.image.load("siyah_at.png"), (tile_size, tile_size)),
    "k": pygame.transform.scale(pygame.image.load("siyah_şah.png"), (tile_size, tile_size)),
}

def draw_board():
    colors = [pygame.Color("white"), pygame.Color("gray")]
    for row in range(8):
        for col in range(8):
            color = colors[(row + col) % 2]
            pygame.draw.rect(screen, color, pygame.Rect(col * tile_size, row * tile_size, tile_size, tile_size))

def draw_pieces(board):
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_str = piece.symbol()
            x = (square % 8) * tile_size
            y = (7 - (square // 8)) * tile_size
            img = piece_images.get(piece_str)
            if img:
                screen.blit(img, (x, y))

def promote_pawn(promotion_square, color):
    options = ["Q", "R", "B", "N"]
    base_x = (promotion_square % 8) * tile_size
    base_y = (7 - (promotion_square // 8)) * tile_size

    overlay_x = max(0, min(base_x - tile_size // 2, width - tile_size * 2))
    overlay_y = max(0, min(base_y - tile_size // 2, height - tile_size * 2))
    
    option_rects = []
    option_positions = []
    for row in range(2):
        for col in range(2):
            rect = pygame.Rect(overlay_x + col * tile_size, overlay_y + row * tile_size, tile_size, tile_size)
            option_positions.append(rect)
    
    for i, rect in enumerate(option_positions):
        option_rects.append((rect, options[i]))
        pygame.draw.rect(screen, pygame.Color("blue"), rect, 3)
        key = options[i] if color == chess.WHITE else options[i].lower()
        img = piece_images.get(key)
        if img:
            img_rect = img.get_rect(center=rect.center)
            screen.blit(img, img_rect.topleft)
    
    pygame.display.flip()
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return "Q"
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mx, my = pygame.mouse.get_pos()
                for rect, piece in option_rects:
                    if rect.collidepoint(mx, my):
                        return piece

def handle_game_over(board):
    font = pygame.font.Font(None, 36)
    text_surface = font.render("Game over. Press Enter to restart.", True, pygame.Color("red"))
    text_rect = text_surface.get_rect(center=(width//2, height//2))
    screen.blit(text_surface, text_rect)
    pygame.display.flip()
    
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                board.reset()
                waiting = False

def highlight_moves(board, square):
    for move in board.legal_moves:
        if move.from_square == square:
            x = (move.to_square % 8) * tile_size
            y = (7 - (move.to_square // 8)) * tile_size
            pygame.draw.circle(screen, pygame.Color("green"), (x + tile_size // 2, y + tile_size // 2), 10)

piece_values = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

def evaluate_board(board):
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = piece_values.get(piece.piece_type, 0)
            score += value if piece.color == chess.WHITE else -value
    return score

def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board), None

    best_move = None
    if maximizing_player:
        max_eval = -float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval_score, _ = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in board.legal_moves:
            board.push(move)
            eval_score, _ = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval, best_move

def log_game_result(board, game_result):
    with open("game_results.txt", "a") as f:
        f.write(f"Result: {game_result}\n")
        f.write(str(board) + "\n\n")

def ai_move(board, depth):
    maximizing = board.turn == chess.WHITE
    dynamic_depth = min(depth, 3) if board.fullmove_number < 10 else depth
    _, move = minimax(board, dynamic_depth, -float('inf'), float('inf'), maximizing)
    return move

def choose_color():
    font = pygame.font.Font(None, 48)
    text = font.render("Press W for White or B for Black", True, pygame.Color("black"))
    rect = text.get_rect(center=(width // 2, height // 2 - 20))
    screen.fill(pygame.Color("white"))
    screen.blit(text, rect)
    pygame.display.flip()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w:
                    return chess.WHITE
                elif event.key == pygame.K_b:
                    return chess.BLACK

def export_game_moves_to_json(game):

    moves_list = []
    board_copy = game.board()
    for move in game.mainline_moves():
        san_move = board_copy.san(move)
        moves_list.append(san_move)
        board_copy.push(move)
    return {"moves": moves_list}

board = chess.Board()
current_game = chess.pgn.Game()
current_game.headers["Event"] = "Training Data"
node = current_game

user_color = choose_color()
ai_color = not user_color

selected_square = None
running = True

while running:
    draw_board()
    draw_pieces(board)
    if selected_square is not None:
        highlight_moves(board, selected_square)
    pygame.display.flip()

    if board.is_game_over():
        # --- Değişiklik: Hamle verilerini JSON formatında dışa aktaralım ---
        game_json = export_game_moves_to_json(current_game)
        with open("game_logs.jsonl", "a", encoding="utf-8") as f:
            json.dump(game_json, f)
            f.write("\n")
        
        if board.is_checkmate():
            game_result = "Checkmate"
        elif board.is_stalemate():
            game_result = "Stalemate"
        elif board.is_insufficient_material():
            game_result = "Insufficient Material"
        elif board.is_seventyfive_moves():
            game_result = "Draw by 75-move rule"
        elif board.is_fivefold_repetition():
            game_result = "Draw by fivefold repetition"
        else:
            game_result = "Unknown"

        log_game_result(board, game_result)
        handle_game_over(board)
        selected_square = None
        current_game = chess.pgn.Game()
        current_game.headers["Event"] = "Training Data"
        node = current_game
        continue

    if board.turn == ai_color:
        pygame.time.wait(500)
        move = ai_move(board, depth=3)
        if move and move in board.legal_moves:
            board.push(move)
            node = node.add_variation(move)
        continue

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            x, y = pygame.mouse.get_pos()
            col = x // tile_size
            row = 7 - (y // tile_size)
            square = chess.square(col, row)

            if selected_square is None:
                if board.piece_at(square) and board.piece_at(square).color == board.turn:
                    selected_square = square
            else:
                move = chess.Move(selected_square, square)
                if board.piece_at(selected_square).piece_type == chess.PAWN and (square < 8 or square >= 56):
                    promotion_piece = promote_pawn(square, board.turn)
                    move = chess.Move(selected_square, square, promotion=chess.Piece.from_symbol(promotion_piece).piece_type)
                if move in board.legal_moves:
                    board.push(move)
                    node = node.add_variation(move)
                selected_square = None

pygame.quit()
sys.exit()